**Install Required Libraries**

In [ ]:
!pip install pypdf langchain_openai langchain_community faiss-cpu google-colab --quiet

**Set Up Your OpenAI API**

In [ ]:
# set openai Key

from google.colab import userdata
import os
# Access the secret and set it as an environment variable
os.environ['OPENAI_API_KEY'] = userdata.get('openai')

Define LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")
llm.invoke("what can you help me with?")

loading  PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("/content/attention is all you need.pdf")
pages= loader.load()

parsing PDF

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter= CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
docs= text_splitter.split_documents(pages)

Load the Documents into a Knowledge Base

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
faiss_index= FAISS.from_documents(pages, OpenAIEmbeddings())
faiss_index.save_local("attention is all you need")

load the saved embeddings

In [ ]:
embeddings= OpenAIEmbeddings()

loaded_vectors= FAISS.load_local("attention is all you need", embeddings, allow_dangerous_deserialization=True)

Rag Chain

In [ ]:
from langchain.chains import ConversationalRetravialChain
from langchain.llms import OpenAI

QA= ConversationalRetravialChain.from_llm(
    llm=llm,
    retriever=loaded_vectors.as_retriever()
)

In [ ]:
chat_history=[]
def rag(query):
  response=QA({"question":query, "chat_history":chat_history})
  chat_history.append((query, response['answer']))
  return response['answer'].strip()

In [ ]:
rag("Can you summarize the key findings of the research?")

In [ ]:
rag("What are the benefits of using multi-head attention?")